In [1]:
from datasetsforecast.long_horizon import LongHorizon, ETTh1

In [2]:
y_df, X_df, _ = LongHorizon.load(directory="data", group="ETTh1")
y_df

,unique_id,ds,y
0,OT,2016-07-01 00:00:00,1.460552
1,OT,2016-07-01 01:00:00,1.161527
2,OT,2016-07-01 02:00:00,1.161527
3,OT,2016-07-01 03:00:00,0.862611
4,OT,2016-07-01 04:00:00,0.525227
...,...,...,...
14395,OT,2018-02-20 19:00:00,-1.866537
14396,OT,2018-02-20 20:00:00,-1.866537
14397,OT,2018-02-20 21:00:00,-1.659595
14398,OT,2018-02-20 22:00:00,-1.628865


In [3]:
X_df

,unique_id,ds,ex_1,ex_2,ex_3,ex_4
0,OT,2016-07-01 00:00:00,-0.500000,0.166667,-0.500000,-0.001370
1,OT,2016-07-01 01:00:00,-0.456522,0.166667,-0.500000,-0.001370
2,OT,2016-07-01 02:00:00,-0.413043,0.166667,-0.500000,-0.001370
3,OT,2016-07-01 03:00:00,-0.369565,0.166667,-0.500000,-0.001370
4,OT,2016-07-01 04:00:00,-0.326087,0.166667,-0.500000,-0.001370
...,...,...,...,...,...,...
14395,OT,2018-02-20 19:00:00,0.326087,-0.333333,0.133333,-0.363014
14396,OT,2018-02-20 20:00:00,0.369565,-0.333333,0.133333,-0.363014
14397,OT,2018-02-20 21:00:00,0.413043,-0.333333,0.133333,-0.363014
14398,OT,2018-02-20 22:00:00,0.456522,-0.333333,0.133333,-0.363014


In [4]:
import pandas as pd


df = X_df.merge(y_df, on=["ds", "unique_id"])
df["ds"] = pd.to_datetime(df["ds"])
df

,unique_id,ds,ex_1,ex_2,ex_3,ex_4,y
0,OT,2016-07-01 00:00:00,-0.500000,0.166667,-0.500000,-0.001370,1.460552
1,OT,2016-07-01 01:00:00,-0.456522,0.166667,-0.500000,-0.001370,1.161527
2,OT,2016-07-01 02:00:00,-0.413043,0.166667,-0.500000,-0.001370,1.161527
3,OT,2016-07-01 03:00:00,-0.369565,0.166667,-0.500000,-0.001370,0.862611
4,OT,2016-07-01 04:00:00,-0.326087,0.166667,-0.500000,-0.001370,0.525227
...,...,...,...,...,...,...,...
14395,OT,2018-02-20 19:00:00,0.326087,-0.333333,0.133333,-0.363014,-1.866537
14396,OT,2018-02-20 20:00:00,0.369565,-0.333333,0.133333,-0.363014,-1.866537
14397,OT,2018-02-20 21:00:00,0.413043,-0.333333,0.133333,-0.363014,-1.659595
14398,OT,2018-02-20 22:00:00,0.456522,-0.333333,0.133333,-0.363014,-1.628865


In [5]:
train_df, test_df = df.iloc[: ETTh1.test_size], df.iloc[ETTh1.test_size :]
train_df.shape, test_df.shape

((11520, 7), (2880, 7))

In [6]:
train_df

,unique_id,ds,ex_1,ex_2,ex_3,ex_4,y
0,OT,2016-07-01 00:00:00,-0.500000,0.166667,-0.500000,-0.001370,1.460552
1,OT,2016-07-01 01:00:00,-0.456522,0.166667,-0.500000,-0.001370,1.161527
2,OT,2016-07-01 02:00:00,-0.413043,0.166667,-0.500000,-0.001370,1.161527
3,OT,2016-07-01 03:00:00,-0.369565,0.166667,-0.500000,-0.001370,0.862611
4,OT,2016-07-01 04:00:00,-0.326087,0.166667,-0.500000,-0.001370,0.525227
...,...,...,...,...,...,...,...
11515,OT,2017-10-23 19:00:00,0.326087,-0.500000,0.233333,0.308219,-0.747264
11516,OT,2017-10-23 20:00:00,0.369565,-0.500000,0.233333,0.308219,-0.808616
11517,OT,2017-10-23 21:00:00,0.413043,-0.500000,0.233333,0.308219,-0.915956
11518,OT,2017-10-23 22:00:00,0.456522,-0.500000,0.233333,0.308219,-0.900591


In [7]:
test_df

,unique_id,ds,ex_1,ex_2,ex_3,ex_4,y
11520,OT,2017-10-24 00:00:00,-0.500000,-0.333333,0.266667,0.310959,-0.862341
11521,OT,2017-10-24 01:00:00,-0.456522,-0.333333,0.266667,0.310959,-0.869969
11522,OT,2017-10-24 02:00:00,-0.413043,-0.333333,0.266667,0.310959,-0.831610
11523,OT,2017-10-24 03:00:00,-0.369565,-0.333333,0.266667,0.310959,-0.877597
11524,OT,2017-10-24 04:00:00,-0.326087,-0.333333,0.266667,0.310959,-0.915956
...,...,...,...,...,...,...,...
14395,OT,2018-02-20 19:00:00,0.326087,-0.333333,0.133333,-0.363014,-1.866537
14396,OT,2018-02-20 20:00:00,0.369565,-0.333333,0.133333,-0.363014,-1.866537
14397,OT,2018-02-20 21:00:00,0.413043,-0.333333,0.133333,-0.363014,-1.659595
14398,OT,2018-02-20 22:00:00,0.456522,-0.333333,0.133333,-0.363014,-1.628865


In [8]:
from neuralforecast import NeuralForecast
from neuralforecast.models import Informer

In [16]:
horizon = ETTh1.horizons[0]

# Try different hyperparmeters to improve accuracy.
models = [
    Informer(
        h=horizon,
        input_size=2 * horizon,
        futr_exog_list=["ex_1", "ex_2", "ex_3", "ex_4"],
        max_steps=10,
    )
]
nf = NeuralForecast(models=models, freq=ETTh1.freq)
nf.fit(train_df)

INFO:lightning_fabric.utilities.seed:Seed set to 1
/Users/irlirion/code/NAS-LLM-Agent/.venv/lib/python3.13/site-packages/neuralforecast/common/_base_model.py:591: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | enc_embedding | DataEmbedding | 896    | train
4 | dec_embedding | DataEmbedding | 896    | train
5 | encoder       | TransEncoder  | 199 K  | train
6 | decoder       | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/Users/irlirion/code/NAS-LLM-Agent/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [55]:
Y_hat = nf.predict(df=Y_df_train, h=len(Y_df_test))
Y_hat

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:44<00:00,  0.02it/s]


,unique_id,ds,Informer
0,HUFL,2017-09-01 00:00:00,10.906538
1,HUFL,2017-09-01 01:00:00,11.264750
2,HUFL,2017-09-01 02:00:00,11.873631
3,HUFL,2017-09-01 03:00:00,11.213528
4,HUFL,2017-09-01 04:00:00,11.283491
...,...,...,...
351423,OT,2023-05-24 15:00:00,10.233512
351424,OT,2023-05-24 16:00:00,10.233512
351425,OT,2023-05-24 17:00:00,10.233512
351426,OT,2023-05-24 18:00:00,10.233511


In [57]:
from utilsforecast.evaluation import evaluate
import utilsforecast.losses as ufl

In [59]:
evaluate(
    Y_df_test.merge(Y_hat),
    metrics=[ufl.mse],
)

,unique_id,metric,Informer
0,HUFL,mse,80.310255
1,HULL,mse,27.384314
2,LUFL,mse,1.673648
3,LULL,mse,0.198368
4,MUFL,mse,80.173198
5,MULL,mse,31.958683
6,OT,mse,23.663228


In [61]:
from utilsforecast.plotting import plot_series